In [2]:
%cd ..

/home/smehta/Projects/genea/CLIP


In [3]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_motion(image)
    text_features = model.encode_inputs(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.9927   0.00412  0.002968]]


In [4]:
text

tensor([[49406,   320, 22697, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [49406,   320,  1929, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0, 

In [5]:
import pandas as pd

In [11]:
a = pd.read_pickle('data/genea2023/trn/main-agent/data2vec-audio/trn_2023_v0_000_main-agent.muted.pkl')
t = pd.read_pickle('data/genea2023/trn/main-agent/data2vec-text/trn_2023_v0_000_main-agent.pkl')
m = pd.read_pickle('data/genea2023/trn/main-agent/diffusion_genea23_sm0_0_30fps/trn_2023_v0_000_main-agent.bvh_expmap_30fps.pkl')


In [12]:
a.shape, t.shape, m.shape

((1950, 768), (1906, 768), (1950, 60))

In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/smehta/Projects/genea/CLIP


In [2]:

import clip
import torch

model = clip.load('lightning_logs/clip_run/version_0/checkpoints/epoch=11-step=1584.ckpt')

/home/smehta/anaconda3/envs/GestCLIP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This model was trained with the following modalities:  ['text', 'audio']
Expected inputs: {'text': (500, 768), 'audio': (500, 768)}


In [3]:
import pandas as pd

In [5]:
x  = {
    'audio': torch.from_numpy(pd.read_pickle('data/genea2023/trn/main-agent/data2vec-audio/trn_2023_v0_000_main-agent.pkl').values),
    'text': torch.from_numpy(pd.read_pickle('data/genea2023/trn/main-agent/data2vec-text/trn_2023_v0_000_main-agent.pkl').values),
}

In [6]:
x['audio'].shape, x['text'].shape

(torch.Size([1950, 768]), torch.Size([1906, 768]))

In [7]:
l = model.embed(x)

In [8]:
l.shape

torch.Size([1950, 512])

In [9]:
y = {
    'motion': torch.from_numpy(pd.read_pickle('data/genea2023/trn/main-agent/diffusion_genea23_sm0_0_30fps/trn_2023_v0_000_main-agent.bvh_expmap_30fps.pkl').values).to('cuda'),
}

In [10]:
model.embed_motion(y).shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model.embed_motion(y).shape                                                                  │
│   2                                                                                              │
│                                                                                                  │
│ /home/smehta/Projects/genea/CLIP/clip/lightning_module.py:304 in embed_motion                    │
│                                                                                                  │
│   301 │   │   for chunked_x in torch.split(batch['motion'], self.hparams.context_length, dim=0   │
│   302 │   │   │   orig_len = chunked_x.shape[0]                                                  │
│   303 │   │   │   inp = F.pad(chunked_x.unsqueeze(0), (0, 0, 0, self.hparams.context_length -    │
│ ❱ 304 │   │   │   emb = self.model.encode_motion(inp)                                            │
│   305 │   │   │   embeddings.append(emb.squeeze(0)[:orig_len])                                   │
│   306 │   │   return torch.concat(embeddings, dim=0)                                             │
│   307                                                                                            │
│                                                                                                  │
│ /home/smehta/Projects/genea/CLIP/clip/model.py:343 in encode_motion                              │
│                                                                                                  │
│   340 │   │                                                                                      │
│   341 │   │   x = motion + pos_emb                                                               │
│   342 │   │   x = x.permute(1, 0, 2)  # NLD -> LND                                               │
│ ❱ 343 │   │   x = self.motion_transformer(x)                                                     │
│   344 │   │   x = x.permute(1, 0, 2)  # LND -> NLD                                               │
│   345 │   │   # x = self.ln_final_motion(x)                                                      │
│   346                                                                                            │
│                                                                                                  │
│ /home/smehta/anaconda3/envs/GestCLIP/lib/python3.10/site-packages/torch/nn/modules/module.py:150 │
│ 1 in _call_impl                                                                                  │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/smehta/Projects/genea/CLIP/clip/model.py:203 in forward                                    │
│                                                                                                  │
│   200 │   │   self.resblocks = nn.Sequential(*[ResidualAtte